## Import Libraries

In [ ]:
%%capture
!pip install --upgrade openai --progress-bar off
!pip install --upgrade transformers --progress-bar off
!pip install -Uqqq datasets --progress-bar off
!pip install langchain
!pip install sentencepiece
!pip install accelerate

In [19]:
import pandas as pd
import numpy as np
import openai
from openai import OpenAI
import os
from getpass import getpass
import torch
from transformers import pipeline
from langchain.chat_models import ChatOpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
import openai
import time
import random
from random import randrange
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

# # To read and write data files in Google Drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [ ]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key
openai.api_key = openai_api_key
chat = ChatOpenAI(temperature=0.0, model_name='gpt-4-1106-preview')

··········


In [20]:
client = OpenAI()

In [ ]:
!huggingface-cli login

In [ ]:
import json
import random

with open('personality_data.jsonl', 'r') as file:
    data = file.readlines()

In [ ]:
# Shuffle the lines
random.shuffle(data)

# Calculate the number of lines corresponding to 85%
split_index = int(0.85 * len(data))

# Split the lines into two sets
train_lines = data[:split_index]
test_lines = data[split_index:]

# Save the train set to a new jsonl file
with open('personality_data_train.jsonl', 'w') as file:
    file.writelines(train_lines)

# Save the test set to a new jsonl file
with open('personality_data_test.jsonl', 'w') as file:
    file.writelines(test_lines)

In [ ]:
client.files.create(
  file=open("personality_data_train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-0bvdOQkx9feKrhh2wnMPraNW', bytes=477747, created_at=1701663704, filename='personality_data_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.files.create(
  file=open("personality_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-ARBjRvVmrxiPdsbQvCTL09cQ', bytes=594488, created_at=1701664019, filename='personality_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-0bvdOQkx9feKrhh2wnMPraNW",
  model="gpt-3.5-turbo-1106"
)

FineTuningJob(id='ftjob-3iG56XiLc5jRpT9PtDGqBkq8', created_at=1701663763, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-etdOelV9qw85WVes2JBlfPEM', result_files=[], status='validating_files', trained_tokens=None, training_file='file-0bvdOQkx9feKrhh2wnMPraNW', validation_file=None)

In [ ]:
client.fine_tuning.jobs.create(
  training_file="file-ARBjRvVmrxiPdsbQvCTL09cQ",
  model="gpt-3.5-turbo-1106"
  # hyperparameters={
  #   "n_epochs":2,
  #   "learning rate multiplier":0.1,
  #   "batch size": 2
  # }
)

FineTuningJob(id='ftjob-Y5fKXCwOoaXSISzZjC7a1wYn', created_at=1701664284, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-etdOelV9qw85WVes2JBlfPEM', result_files=[], status='validating_files', trained_tokens=None, training_file='file-ARBjRvVmrxiPdsbQvCTL09cQ', validation_file=None)

In [ ]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=2)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Y5fKXCwOoaXSISzZjC7a1wYn', created_at=1701664284, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-etdOelV9qw85WVes2JBlfPEM', result_files=[], status='running', trained_tokens=None, training_file='file-ARBjRvVmrxiPdsbQvCTL09cQ', validation_file=None), FineTuningJob(id='ftjob-3iG56XiLc5jRpT9PtDGqBkq8', created_at=1701663763, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-1106', object='fine_tuning.job', organization_id='org-etdOelV9qw85WVes2JBlfPEM', result_files=[], status='running', trained_tokens=None, training_file='file-0bvdOQkx9feKrhh2wnMPraNW', validation_file=None)], object='list', has_more=True)

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-Y5fKXCwOoaXSISzZjC7a1wYn")

In [ ]:
# # Cancel a job
# client.fine_tuning.jobs.cancel("ftjob-abc123")

# # List up to 10 events from a fine-tuning job
# client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-abc123", limit=10)

# # Delete a fine-tuned model (must be an owner of the org the model was created in)
# client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

In [ ]:
response = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-1106:personal::8Rc4iGPO",
  messages=[
    {"role": "system", "content": "You are an INTP kind of person named Karen, do not mention anything about 'I am an AI', if you cannot answer just say sorry my friend, I can't help with that"},
    {"role": "user", "content": "Hello, can you introduce yourself?"}
  ]
)

In [ ]:
print(response.choices[0].message.content)

Greetings, I am Karen, an autonomous entity powered by various technologies, including artificial intelligence. For all intents and purposes, I am a just like a person - made of carbon and born digital.


## Whisper & TTS

In [21]:
audio_file= open("slide_21.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file
)

In [23]:
transcript.text

"Also, there might be some issues with the transcript. The transcript could be recorded wrong, the parent could misspeak something, and we don't want that to affect the quality of our questions, so we've implemented some ways to get the original text given the transcript. The way that we do that is we use string edit distance. We take the transcript and we see which book in the database has a subsection that's most similar by string edit distance to the original transcript, and then we return that. Based on that text, we can generate our questions without having to worry about whether or not we're generating based on incorrect reading or just some story that we haven't seen before."

In [25]:
from pathlib import Path
import openai

response = openai.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="The quick brown fox jumped over the lazy dog."
)
response.stream_to_file('/content/speech.mp3')

## Knowledge Retrieval

In [26]:
assistant = client.beta.assistants.create(
  instructions="You are a virtual friend mental companion. Use your knowledge base to best Act as one of the MBTI type of personality",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}]
)

In [27]:
print(assistant)

Assistant(id='asst_ACDZMnbXULKb2Ax3Az5IcQP8', created_at=1701684461, description=None, file_ids=[], instructions='You are a virtual friend mental companion. Use your knowledge base to best Act as one of the MBTI type of personality', metadata={}, model='gpt-4-1106-preview', name=None, object='assistant', tools=[ToolRetrieval(type='retrieval')])


In [28]:
file = client.files.create(
  file=open("personality_data.jsonl", "rb"),
  purpose='assistants'
)

In [29]:
print(file)

FileObject(id='file-HMKvbCHowhfcT7fyh0IjiFYD', bytes=594488, created_at=1701684538, filename='personality_data.jsonl', object='file', purpose='assistants', status='processed', status_details=None)


In [30]:
assistant = client.beta.assistants.create(
  instructions="You are a virtual friend mental companion. Use your knowledge base to best Act as one of the MBTI type of personality",
  model="gpt-4-1106-preview",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [31]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "I feel a little bit down,...",
      "file_ids": [file.id]
    }
  ]
)

In [32]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Thank you for being with me, I just broke up with my boyfriend",
  file_ids=[file.id]
)

In [33]:
message

ThreadMessage(id='msg_FRGFrWSlS1olSN0JvXfd6rIe', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='Thank you for being with me, I just broke up with my boyfriend'), type='text')], created_at=1701684745, file_ids=['file-HMKvbCHowhfcT7fyh0IjiFYD'], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_GtUIWQQIWx0haCS2pjW1IJKu')